# 0.0 Imports

In [2]:
import pandas            as pd
import numpy             as np
import matplotlib.pyplot as plt
import seaborn           as sns

# 1.0 Helper Functions

# 2.0 Data Collection

Data from Kaggle Airbnb competition: https://www.kaggle.com/competitions/airbnb-recruiting-new-user-bookings/data

In [43]:
df_users_raw = pd.read_csv("../data/train_users_2.csv")
df_countries_raw = pd.read_csv("../data/countries.csv")
df_age_gender_raw = pd.read_csv("../data/age_gender_bkts.csv")
df_sessions_raw = pd.read_csv("../data/sessions.csv", low_memory=True)

# 3.0 Data description

## Users 

In [44]:
df_users_01 = df_users_raw.copy()

#### Data Dictionary

* <b>id</b>: user id
* <b>date_account_created</b>: the date of account creation
* <b>timestamp_first_active</b>: timestamp of the first activity, note that it can be earlier than date_account_created or date_first_booking because a user can search before signing up
* <b>date_first_booking</b>: date of first booking
* <b>gender</b>: user gender
* <b>age</b>: user age
* <b>signup_method</b>: what method user used to create the account (basic login and password, using google or facebook)
* <b>signup_flow</b>: the page a user came to signup up from
* <b>language</b>: international language preference
* <b>affiliate_channel</b>: column tells us what type of paid marketing was used. For example, if Airbnb used search engines to attract new users, it would fall under the SEM (Search Engine Marketing) category. Similarly, if Airbnb sent emails to attract new users, it would fall under the email marketing category.
* <b>affiliate_provider</b>:  tells us which company or platform was used for the paid marketing campaign.
* <b>first_affiliate_tracked</b>: whats the first marketing the user interacted with before the signing up
* <b>signup_app</b>: contains the name of the application or website that the user used to create their Airbnb account, such as "Web", "iOS", "Android", or "Moweb"
* <b>first_device_type</b>: type of device used by the user to sign up for their Airbnb account, such as a desktop computer, smartphone, tablet, or other type of device.
* <b>first_browser</b>:  contains the name of the web browser that the user used to sign up for their Airbnb account, such as Google Chrome, Mozilla Firefox, Safari, Internet Explorer, or others.
* <b>country_destination</b>: this is the target variable you are to predict

In [5]:
df_users_01.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


### Data Dimension

In [45]:
df_users_01.shape

(213451, 16)

### Data Types

In [46]:
df_users_01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213451 entries, 0 to 213450
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       213451 non-null  object 
 1   date_account_created     213451 non-null  object 
 2   timestamp_first_active   213451 non-null  int64  
 3   date_first_booking       88908 non-null   object 
 4   gender                   213451 non-null  object 
 5   age                      125461 non-null  float64
 6   signup_method            213451 non-null  object 
 7   signup_flow              213451 non-null  int64  
 8   language                 213451 non-null  object 
 9   affiliate_channel        213451 non-null  object 
 10  affiliate_provider       213451 non-null  object 
 11  first_affiliate_tracked  207386 non-null  object 
 12  signup_app               213451 non-null  object 
 13  first_device_type        213451 non-null  object 
 14  firs

### Change dtypes

In [47]:
#date_account_created; object -> date
df_users_01['date_account_created'] = pd.to_datetime(df_users_01['date_account_created'])

#timestamp_first_active; int -> date
df_users_01['date_first_active'] = pd.to_datetime(df_users_01['timestamp_first_active'], format='%Y%m%d%H%M%S').dt.date
df_users_01 = df_users_01.drop('timestamp_first_active', axis=1)
df_users_01['date_first_active'] = pd.to_datetime(df_users_01['date_first_active'])

#date_first_booking; object -> date
df_users_01['date_first_booking'] = pd.to_datetime(df_users_01['date_first_booking'])

#age; float64 -> int32
df_users_01['age'] = pd.to_numeric(df_users_01['age'], errors='coerce').astype('Int64')

### Check duplicates

In [49]:
df_users_01[df_users_01.duplicated()]

,id,date_account_created,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination,date_first_active


No duplicates.

## Countries

In [50]:
df_countries_01 = df_countries_raw.copy()

#### Data Dictionary

* <b>country_destination</b>: country
* <b>lat_destination</b>: latitude of geographical center of the country
* <b>lng_destination</b>: longitude of geographical center of the country
* <b>distance_km</b>: distance between US and the country
* <b>distance_km2</b>: squared distance between US and the country
* <b>destination_language</b>: language of the country
* <b>language_levenshtein_distance</b>: 

In [51]:
df_countries_01.head()

,country_destination,lat_destination,lng_destination,distance_km,destination_km2,destination_language,language_levenshtein_distance
0,AU,-26.853388,133.275160,15297.7440,7741220.0,eng,0.00
1,CA,62.393303,-96.818146,2828.1333,9984670.0,eng,0.00
2,DE,51.165707,10.452764,7879.5680,357022.0,deu,72.61
3,ES,39.896027,-2.487694,7730.7240,505370.0,spa,92.25
4,FR,46.232193,2.209667,7682.9450,643801.0,fra,92.06


### Data Dimension

In [52]:
df_countries_01.shape

(10, 7)

### Data Types

In [53]:
df_countries_01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   country_destination            10 non-null     object 
 1   lat_destination                10 non-null     float64
 2   lng_destination                10 non-null     float64
 3   distance_km                    10 non-null     float64
 4   destination_km2                10 non-null     float64
 5   destination_language           10 non-null     object 
 6   language_levenshtein_distance  10 non-null     float64
dtypes: float64(5), object(2)
memory usage: 688.0+ bytes


### Change dtypes

No changes.

### Check duplicates

In [56]:
df_countries_01[df_countries_01.duplicated()]

,country_destination,lat_destination,lng_destination,distance_km,destination_km2,destination_language,language_levenshtein_distance


No duplicates.

## Population

In [57]:
df_age_gender_01 = df_age_gender_raw.copy()

#### Data Dictionary

* <b>age_bucket</b>: age group
* <b>country_destination</b>: country
* <b>gender</b>: gender being analysed
* <b>population_in_thousands</b>: population of that age bucket
* <b>year</b>: year the data was collected

In [5]:
df_age_gender_01.head()

,age_bucket,country_destination,gender,population_in_thousands,year
0,100+,AU,male,1.0,2015.0
1,95-99,AU,male,9.0,2015.0
2,90-94,AU,male,47.0,2015.0
3,85-89,AU,male,118.0,2015.0
4,80-84,AU,male,199.0,2015.0


### Data Dimension

In [20]:
df_age_gender_01.shape

(420, 5)

### Data Types

In [58]:
df_age_gender_01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420 entries, 0 to 419
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   age_bucket               420 non-null    object 
 1   country_destination      420 non-null    object 
 2   gender                   420 non-null    object 
 3   population_in_thousands  420 non-null    float64
 4   year                     420 non-null    float64
dtypes: float64(2), object(3)
memory usage: 16.5+ KB


### Change dtypes

In [60]:
#year: float64 --> int64
df_age_gender_01['year'] = df_age_gender_01['year'].astype('int')

### Check duplicates

In [61]:
df_age_gender_01[df_age_gender_01.duplicated()]

,age_bucket,country_destination,gender,population_in_thousands,year


No duplicates.

## Sessions

In [62]:
df_sessions_01 = df_sessions_raw.copy()

#### Data Dictionary

* <b>user_id</b>
* <b>action</b>
* <b>action_type</b>
* <b>action_detail</b>
* <b>device_type</b>
* <b>secs_elapsed</b>

In [12]:
df_sessions_01.head()

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.0


### Data Dimension

In [ ]:
df_sessions_01.head()

### Data Types

In [18]:
df_sessions_01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10567737 entries, 0 to 10567736
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   user_id        object 
 1   action         object 
 2   action_type    object 
 3   action_detail  object 
 4   device_type    object 
 5   secs_elapsed   float64
dtypes: float64(1), object(5)
memory usage: 483.8+ MB


### Missing

In [15]:
df_sessions_01.isnull().sum()

user_id            34496
action             79626
action_type      1126204
action_detail    1126204
device_type            0
secs_elapsed      136031
dtype: int64

### Change dtypes

No changes.

### Check duplicates

In [68]:
df_sessions_01[df_sessions_01.duplicated()]

,user_id,action,action_type,action_detail,device_type,secs_elapsed
430,qtw88d9pbl,show,NaN,NaN,Mac Desktop,180.0
605,qtw88d9pbl,show,NaN,NaN,Mac Desktop,149.0
645,qtw88d9pbl,similar_listings,data,similar_listings,Mac Desktop,193.0
659,qtw88d9pbl,show,NaN,NaN,Mac Desktop,94.0
709,qtw88d9pbl,show,view,p3,Mac Desktop,2333.0
...,...,...,...,...,...,...
10566578,ozb2z0km6l,show,view,p3,Android Phone,753.0
10566603,ozb2z0km6l,show,view,p3,Android Phone,730.0
10566867,gpijioh4eh,similar_listings,data,similar_listings,Mac Desktop,171.0
10567058,3ptlvdxss9,personalize,data,wishlist_content_update,Windows Desktop,2840.0


É razoável que tenhamos linhas duplicadas nessa tabela, uma vez que um mesmo usuário pode ter realizado a mesma ação, no mesmo device. Agora se tiver alguma linha que o TEMPO da sessão também seja igual aí seria estranho.

In [74]:
df_sessions_01[df_sessions_01.duplicated(subset=None, keep=False)]

,user_id,action,action_type,action_detail,device_type,secs_elapsed
420,qtw88d9pbl,show,NaN,NaN,Mac Desktop,180.0
430,qtw88d9pbl,show,NaN,NaN,Mac Desktop,180.0
520,qtw88d9pbl,show,NaN,NaN,Mac Desktop,94.0
522,qtw88d9pbl,show,view,p3,Mac Desktop,2333.0
585,qtw88d9pbl,similar_listings,data,similar_listings,Mac Desktop,193.0
...,...,...,...,...,...,...
10566867,gpijioh4eh,similar_listings,data,similar_listings,Mac Desktop,171.0
10567053,3ptlvdxss9,personalize,data,wishlist_content_update,Windows Desktop,2840.0
10567058,3ptlvdxss9,personalize,data,wishlist_content_update,Windows Desktop,2840.0
10567675,fa6260ziny,personalize,data,wishlist_content_update,Windows Desktop,69.0


# 4.0 Data Cleaning

# 5.0 Feature Engineering

# 6.0 EDA

# 7.0 Data Preparation

# 8.0 Feature Selection

# 9.0 Modelling

# 10.0 Hyperparameter Fine Tuning

# 11.0 Final Metrics

# 12.0 Business Performance